In [1]:
#jupyter labextension install @jupyter-widgets/jupyterlab-manager

import ipywidgets as widgets
from IPython.display import display
from queue import Queue
import pyaudio
from queue import Queue
from threading import Thread
from time import sleep
import numpy as np




messages = Queue()
recordings = Queue()

CHANNELS = 1
FRAME_RATE = 16000
RECORD_SECONDS = 0.1
AUDIO_FORMAT = pyaudio.paInt16
SAMPLE_SIZE = 2


record_button = widgets.Button(
    description='Record',
    disabled=False,
    button_style='success',
    tooltip='Record',
    icon='microphone'
)

stop_button = widgets.Button(
    description='Stop',
    disabled=False,
    button_style='warning',
    tooltip='Stop',
    icon='stop'
)

audioDevices=[]
p = pyaudio.PyAudio()
for i in range(p.get_device_count()):
    audioDevices.append((p.get_device_info_by_index(i)['name'],i))
p.terminate()   

deviceList=widgets.Dropdown(
    options=audioDevices,
    value=1,
    description='Audio Device:',
)

volBar=widgets.FloatProgress(
    value=0,
    min=0,
    max=65536,
    description='Volume:',
    bar_style='info',
    style={'bar_color': '#ffff00'},
    orientation='horizontal'
)

output = widgets.Output()


def record_microphone(chunk=1024):
    p = pyaudio.PyAudio()



    stream = p.open(format=AUDIO_FORMAT,
                    channels=CHANNELS,
                    rate=FRAME_RATE,
                    input=True,
                    input_device_index=deviceList.value,
                    frames_per_buffer=chunk)
    
    frames = []

    while not messages.empty():
        data = stream.read(chunk)
        frames.append(data)
        if len(frames) >= (FRAME_RATE * RECORD_SECONDS) / chunk:
            recordings.put(frames.copy())
            dt = np.dtype(np.int16)
            dt = dt.newbyteorder('<')
            for e in frames:
                v=np.frombuffer(e, dtype=dt)
                v=np.mean(np.abs(v))
                volBar.value=v
            frames = []

    stream.stop_stream()
    stream.close()
    p.terminate()
    print("Section done")


def start_recording(data):
    messages.put(True)
    with output:
        print("Starting...")
        print(deviceList.value)
    record = Thread(target=record_microphone)
    record.start()


def stop_recording(data):
    with output:
        messages.get()
        print("Stopped.")


record_button.on_click(start_recording)
stop_button.on_click(stop_recording)


            
display(deviceList,record_button, volBar, stop_button, output)


Dropdown(description='Audio Device:', index=1, options=(('Microsoft Sound Mapper - Input', 0), ('Microphone (R…

Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle(), tooltip='Record')

FloatProgress(value=0.0, bar_style='info', description='Volume:', max=65536.0, style=ProgressStyle(bar_color='…

Button(button_style='warning', description='Stop', icon='stop', style=ButtonStyle(), tooltip='Stop')

Output()